## 4. Extra Topic: Convolutional Neural Networks

We define a new Deep Learning variant called **1D Convolutional Neural Network (CNN)** below.
CNN is a type of deep learning model that is typically used for image and signal processing tasks. 

In this code, the model takes a 1D input signal of length 1024 and performs a series of convolution, pooling and fully connected operations to classify the signal into one of two classes. The `conv1` and `conv2` layers are 1D convolutional layers that apply filters to the input signal to extract features. The `pool1` and `pool2` layers are max-pooling layers that down-sample the input signal. The `fc1` and `fc2` layers are fully connected layers that perform classification based on the extracted features. The `softmax` activation is applied to the output to produce probabilities for each class. This gives us the classification into soluble and insoluble molecules. 

The steps are similar to DNN model. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the 1D CNN model
class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 256, 128)
        self.fc2 = nn.Linear(128, 2)
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        x = x.reshape(-1, 1, 1024)
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.reshape(-1, 32 * 256)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.softmax(x)
        return x


In [ ]:
# Initialize the model, loss function, and optimizer
model_cnn = CNN1D()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_cnn.parameters(), lr=0.01)

In [ ]:
summary(model_cnn)

In [ ]:
# Train the CNN model and keep track of validation loss
train_loss1 = []
val_loss1 = []
num_epochs = 50

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model_cnn(inputs)
        
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    avg_train_loss = running_loss / len(trainloader)
    train_loss1.append(avg_train_loss)
    
    val_running_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(valloader, 0):
            inputs, labels = data
            outputs = model_cnn(inputs)
            
            val_batch_loss = criterion(outputs, labels.long())
            val_running_loss += val_batch_loss.item()
        
    avg_val_loss = val_running_loss / len(valloader)
    val_loss1.append(avg_val_loss)
    

In [ ]:
# Plot the validation loss
plt.plot(range(num_epochs), train_loss1, label='Training Loss')
plt.plot(range(num_epochs), val_loss1, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

<div class="alert alert-success">
<b>Task 4a.</b> From the above plot, what do you think about the CNN model's learning over 50 epochs. Compare its performance with DNN. </div>

In [ ]:
# Test the model on the test data
correct = 0
total = 0
predictions = []
true_labels = []
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        inputs, target = data
        outputs = model_cnn(inputs)
        _, predicted = torch.max(outputs.data, 1) # Get the class with the highest probability
        total += target.size(0)
        correct += (predicted == target.long()).sum().item() # Convert target to long and compare
        predictions += predicted.tolist()
        true_labels += target.tolist()


In [ ]:
accuracy = correct / total
print(f'Accuracy of the network on the test data: {(100 * accuracy):.2f} %' )

In [ ]:
# Plot confusion matrix
cf_matrix= confusion_matrix(true_labels, predictions)
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')



<div class="alert alert-success">
<b>Task 4b.</b> Is there a change in true positive, false positive and false negative rates as compared to the DNN model? </div>